## Notebook para Execução da reconfiguração parcial com vários cores

### Elaborado por: Téo Sobrino Alves
### Feito sob licença GPL 3.0
#### Mais detalhes: https://github.com/TeOSobrino/IC_code/tree/main

In [1]:
from pynq.overlays.base import BaseOverlay
from pynq import Overlay
import time

In [2]:
# carrega o overlay base da fpga - permite interagir com leds, botões, etc
base = BaseOverlay("base.bit")
base_path = '/home/xilinx/'
current_ol = 'none'

In [ ]:
# carrega o bitstream base principal do processador
def load_main_ol():
    ol = Overlay(base_path + 'multicore_pl_xilinx_zcu104.bit ')
    return ol

In [ ]:
# devemos declarar qual região da FPGA foi marcada como reconfigurável com o 
# nome do bdf parcial usado naquele design, em nosso design temos 3 blocos
# no_fpu_extra_core; remove_core_inc_cache; no_fpu_inc_cache;

# bloco que retira a FPU e adiciona um core
def set_pr_remove_FPU_add_core(current_ol: list, ol:PynqOverlay):
    if(ol == None):
        ol = load_main_ol()
    Overlay.set_partial_region('no_fpu_extra_core')
    current_ol = "no_fpu_extra_core"

# bloco que remove um core e aumenta a cache dos demais
def set_pr_remove_core_inc_cache(current_ol: list, ol:PynqOverlay):
    if(ol == None):
        ol = load_main_ol()
    Overlay.set_partial_region('remove_core_inc_cache')
    current_ol = "remove_core_inc_cache"
    return ol, current_ol

# bloco que remove a FPU e aumenta a cache dos demais
def set_pr_no_fpu_inc_cache(current_ol: list, ol:PynqOverlay):
    if(ol == None):
        ol = load_main_ol()
    Overlay.set_partial_region('no_fpu_inc_cache')
    current_ol = "no_fpu_inc_cache"
    return ol, current_ol

#resetamos a arquitetura para a estática inicial
def reset_arch(current_ol: list, ol:PynqOverlay):
    if(ol == None):
        ol = load_main_ol()
    current_ol = None
    return ol, current_ol


In [ ]:
# precisamos direcionar o processador para o arquivo RISCV que executaremos (testbench)
ol = load_main_ol()
path = "home/Xilinx/teo/tb/asm/drnn"
found = ol.bp_set_program_path(path)
if not found:
    print(f"Arquivo não encontrado em (File not found at): {path}")
    exit

In [ ]:
def read_triggers(ol: PynqOverlay):
    t1 = ol.bp_read_register('cache_rate')
    t2 = ol.bp_read_register('FPU_access')
    return (t1,t2)

In [ ]:
#lemos a taxa de hit/miss do processador BlackParrot
trigger = read_triggers(ol)

In [ ]:
# a depender dos valores atuais das variáveis, fazemos a troca das bitstreams
trigger_val = 0.7
exchange = 0

def load_ol_trigger(program_status:int, ol: PynqOverlay):
    while not program_status and not exchange:
        if trigger >= trigger_val:
            if current_ol != "mem_inc":
                set_pr_mem_inc()
            Overlay.download(base_path + 'xilinx_zcu104_remove_core_extra_cache.bit')
        elif trigger <= (1-trigger_val):
            if current_ol != "mem_dec":
                set_pr_mem_inc()
            Overlay.download(base_path + 'xilinx_zcu104_reduced_cache_two_fpu.bit')
        elif trigger_val == 0.5:
            Overlay.download(base_path + 'xilinx_zcu104_remove_core_extra_cache.bit')
        # esperamos para realizar a leitura do registrador
        
        trigger = ol.bp_read_register('cache_rate')

        

In [ ]:
program_status = 0
ol.bp_begin_assync_execution(program_status)
load_ol_trigger(program_status, ol)